In [1]:
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma


In [2]:
files = [
    "files/apostila.pdf",
    "files/LLM.pdf"
]
pages = []
for file in files:
    loader = PyMuPDFLoader(file)
    pages.extend(loader.load())


chunk_size = 1000
chunk_overlap = 100

recursive_split = RecursiveCharacterTextSplitter(
    chunk_overlap=chunk_overlap,
    chunk_size=chunk_size,
    separators=["\n\n","\n"," ", "", "."]
)
documents = recursive_split.split_documents(pages)

for i, doc in enumerate(documents):
    doc.metadata["source"] = doc.metadata["source"].replace("files/","")
    doc.metadata["doc_id"] = i

In [3]:
directory = "files/chroma_retrival_db" 

embeddings_model = OpenAIEmbeddings()
vectordb= Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory= directory
)

In [4]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")

In [5]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever= vectordb.as_retriever(search_type="mmr")
)

In [6]:
question = "O que é Hugging Face e como faço para acessa-lo?"

chat_chain.invoke({"query":question})

{'query': 'O que é Hugging Face e como faço para acessa-lo?',
 'result': 'Hugging Face é uma plataforma e empresa conhecida por seus modelos de aprendizado de máquina, especialmente em processamento de linguagem natural (NLP). A Hugging Face oferece uma biblioteca chamada Transformers, que permite aos desenvolvedores utilizar e ajustar modelos de linguagem pré-treinados em suas aplicações. \n\nPara acessar o Hugging Face, você pode visitar o site oficial em [huggingface.co](https://huggingface.co). A partir daí, você pode explorar os modelos disponíveis, usar a API da Hugging Face ou instalar a biblioteca Transformers em seu ambiente Python para trabalhar com modelos como o BERT, GPT-2, e outros. \n\nVocê pode instalar a biblioteca usando o comando:\n\n```bash\npip install transformers\n```\n\nDepois de instalada, você pode começar a usar os modelos como objetos Python em seu código.'}

In [8]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
    """
    Utilize o contexto forncedido para responder a pergunta ao final.
    Se você não souber a resposta, apenas diga que não sabe e não invente uma resposta.
    Utilize três frases no maximo, matenha a resposta concisa.

    Contexto: {context}
    Pergunta: {question}

    Resposta:
"""
)

In [9]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever= vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":chain_prompt},
    return_source_documents=True
)

In [10]:
question = "Como a apostila mostra para usar o ChatGPT?"

response = chat_chain.invoke({"query":question})
print(response["result"])

A apostila oferece códigos de exemplo que ajudam os usuários a começar a trabalhar com LLMs, incluindo o ChatGPT. Ela é projetada para cursos de extensão de Introdução à Programação com Python, facilitando a compreensão e a aplicação prática. Além disso, o material foi desenvolvido por um grupo de estudantes para auxiliar no aprendizado.


In [12]:
response["source_documents"]

[Document(metadata={'modDate': "D:20231017132635Z00'00'", 'file_path': 'files/LLM.pdf', 'format': 'PDF 1.4', 'author': '', 'producer': 'macOS Version 13.5 (Build 22G74) Quartz PDFContext', 'subject': '', 'source': 'LLM.pdf', 'creationDate': "D:20231017132635Z00'00'", 'keywords': '', 'trapped': '', 'creator': '', 'total_pages': 9, 'moddate': "D:20231017132635Z00'00'", 'title': '', 'doc_id': 115, 'page': 5, 'creationdate': "D:20231017132635Z00'00'"}, page_content='Serviços proprietários \nComo o primeiro serviço amplamente disponível alimentado por LLM, o \nChatGPT da OpenAI foi o catalisador explosivo que trouxe os LLMs para o \nmainstream. O ChatGPT fornece uma interface de usuário (ou API) em que os \nusuários podem enviar prompts para muitos \nmodelos (GPT-3.5, GPT-4 e outros) e geralmente obter uma resposta rápida. Eles \nestão entre os modelos de maior desempenho, treinados em conjuntos de dados'),
 Document(metadata={'creationDate': "D:20160504100639-03'00'", 'creator': 'Microsoft

### Como debugar a cadeia

In [13]:
from langchain.globals import set_debug

set_debug(True)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})

set_debug(False)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "O que é Hugging Face e como faço para acessá-lo?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "O que é Hugging Face e como faço para acessá-lo?",
  "context": "modelo transformador da Hugging Face e usá-lo como um objeto Python. Muitas \nvezes, você pode encontrar um modelo de código aberto que resolve seu \nproblema específico e que é várias ordens de grandeza menor que o ChatGPT, \npermitindo que você traga o modelo para seu ambiente e hospede-o você \nmesmo. Isso significa que você pode manter os dados sob seu controle para \npreocupações com privacidade e governança, além de gerenciar seus custos.\n\nSobre a Databricks \nA Databricks é a empresa de dados e IA. Mais de 9.000 \norganizações em todo o mundo, incluindo a Comc

In [ ]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever= vectordb.as_retriever(search_type="mmr"),
    chain_type="refine"
)

pergunta = 'O que é Hugging Face e como faço para acessá-lo?'
resposta = chat_chain.invoke({'query': pergunta})
print(response["result"])